In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset
import math
from sklearn.model_selection import train_test_split
import warnings
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import metrics
from sklearn.decomposition import PCA

In [ ]:
LIMIT_FOR_LABEL_ENCODING = 2
N_EPOCHS=100
BATCH_SIZE=247

def fillNA(dataset):
  for column in tqdm(dataset.columns[dataset.dtypes!='object']):
    if(dataset[column].isnull().values.any()):
      #dataset[column].fillna(dataset[column].quantile(0.75), inplace=True)
      dataset[column].fillna(dataset[column].median(), inplace=True)
  return dataset

def encode_ftrs(cat, dataset):
  le = LabelEncoder()
  le_count = 0
  all_ftrs = dataset.select_dtypes(include=['object']).shape[1]
  for col in cat:
      if (dataset[col].dtype == 'object'):
          if (len(list(dataset[col].unique())) <= LIMIT_FOR_LABEL_ENCODING):
              le.fit(dataset[col])
              dataset[col] = le.transform(dataset[col])
              le_count += 1
  print(f'%d columns were label encoded.' % le_count)

  dataset = pd.get_dummies(dataset)
  mns = all_ftrs-le_count
  print(f'%d columns were one-hot encoded.' % mns)
  return dataset

def create_tensor_ds(X_train, y_train, X_test, y_test):
  X_tr_tnsr = torch.tensor(X_train.values, dtype=torch.float32)
  y_tr_tnsr = torch.tensor(y_train.values, dtype=torch.float32)
  train_ds = TensorDataset(X_tr_tnsr, y_tr_tnsr)

  X_tst_tnsr = torch.tensor(X_test.values, dtype=torch.float32)
  y_tst_tnsr = torch.tensor(y_test.values, dtype=torch.float32)
  test_ds = TensorDataset(X_tst_tnsr, y_tst_tnsr)
  
  return train_ds, test_ds

def get_predictions(loader, model, device):
  model.eval()
  saved_preds = []
  true_labels = []
  
  with torch.no_grad():
      for x,y in loader:
          x = x.to(device)
          y = y.to(device)
          scores = model(x)
          saved_preds += scores.tolist()
          true_labels += y.tolist()
  #model.train()
  return saved_preds, true_labels

def train_model_m(n_hidden_neurons, NN, device):
  model = NN(input_size=X.shape[1], n_hidden_neurons=n_hidden_neurons).to(device)
  loss_fn = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.00025, weight_decay=1e-4)

  for epoch in range(N_EPOCHS):
    proba, true = get_predictions(test_loader_m, model, device=device)
    print(f"MSE: {metrics.mean_squared_error(true, proba)}")
    for batch_idx, (data, targets) in enumerate(train_loader_m):
      data = data.to(device)
      targets = targets.to(device)

      scores = model(data)
      loss = loss_fn(scores, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  return model

def train_model_d(n_hidden_neurons, NN, device):
  model = NN(input_size=X.shape[1], n_hidden_neurons=n_hidden_neurons).to(device)
  loss_fn = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.00025, weight_decay=1e-4)

  for epoch in range(N_EPOCHS):
    proba, true = get_predictions(test_loader_d, model, device=device)
    print(f"MSE: {metrics.mean_squared_error(true, proba)}")
    for batch_idx, (data, targets) in enumerate(train_loader_d):
      data = data.to(device)
      targets = targets.to(device)

      scores = model(data)
      loss = loss_fn(scores, targets)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
  return model

In [ ]:
class NN(nn.Module):
  def __init__(self, input_size, n_hidden_neurons):
    super(NN, self).__init__()

    self.fc1 = nn.Linear(input_size, n_hidden_neurons[0])
    self.act1 = torch.nn.ReLU()
    self.fc2 = nn.Linear(n_hidden_neurons[0], 1)

  def forward(self, x):
    x = self.fc1(x)
    x = self.act1(x)
    x = self.fc2(x)
    return x.reshape(-1)

In [ ]:
data = pd.read_csv("database.csv")

In [ ]:
data

,Date,Time,Latitude,Longitude,Type,Depth,Depth Error,Depth Seismic Stations,Magnitude,Magnitude Type,...,Magnitude Seismic Stations,Azimuthal Gap,Horizontal Distance,Horizontal Error,Root Mean Square,ID,Source,Location Source,Magnitude Source,Status
0,01/02/1965,13:44:18,19.2460,145.6160,Earthquake,131.60,NaN,NaN,6.0,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860706,ISCGEM,ISCGEM,ISCGEM,Automatic
1,01/04/1965,11:29:49,1.8630,127.3520,Earthquake,80.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860737,ISCGEM,ISCGEM,ISCGEM,Automatic
2,01/05/1965,18:05:58,-20.5790,-173.9720,Earthquake,20.00,NaN,NaN,6.2,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860762,ISCGEM,ISCGEM,ISCGEM,Automatic
3,01/08/1965,18:49:43,-59.0760,-23.5570,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860856,ISCGEM,ISCGEM,ISCGEM,Automatic
4,01/09/1965,13:32:50,11.9380,126.4270,Earthquake,15.00,NaN,NaN,5.8,MW,...,NaN,NaN,NaN,NaN,NaN,ISCGEM860890,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,12/28/2016,08:22:12,38.3917,-118.8941,Earthquake,12.30,1.2,40.0,5.6,ML,...,18.0,42.47,0.120,NaN,0.1898,NN00570710,NN,NN,NN,Reviewed
23408,12/28/2016,09:13:47,38.3777,-118.8957,Earthquake,8.80,2.0,33.0,5.5,ML,...,18.0,48.58,0.129,NaN,0.2187,NN00570744,NN,NN,NN,Reviewed
23409,12/28/2016,12:38:51,36.9179,140.4262,Earthquake,10.00,1.8,NaN,5.9,MWW,...,NaN,91.00,0.992,4.8,1.5200,US10007NAF,US,US,US,Reviewed
23410,12/29/2016,22:30:19,-9.0283,118.6639,Earthquake,79.00,1.8,NaN,6.3,MWW,...,NaN,26.00,3.553,6.0,1.4300,US10007NL0,US,US,US,Reviewed


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23412 entries, 0 to 23411
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Date                        23412 non-null  object 
 1   Time                        23412 non-null  object 
 2   Latitude                    23412 non-null  float64
 3   Longitude                   23412 non-null  float64
 4   Type                        23412 non-null  object 
 5   Depth                       23412 non-null  float64
 6   Depth Error                 4461 non-null   float64
 7   Depth Seismic Stations      7097 non-null   float64
 8   Magnitude                   23412 non-null  float64
 9   Magnitude Type              23409 non-null  object 
 10  Magnitude Error             327 non-null    float64
 11  Magnitude Seismic Stations  2564 non-null   float64
 12  Azimuthal Gap               7299 non-null   float64
 13  Horizontal Distance         160

In [ ]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Latitude,23412.0,1.679033,30.113183,-77.080000,-18.65300,-3.5685,26.19075,86.005
Longitude,23412.0,39.639961,125.511959,-179.997000,-76.34975,103.9820,145.02625,179.998
Depth,23412.0,70.767911,122.651898,-1.100000,14.52250,33.0000,54.00000,700.000
Depth Error,4461.0,4.993115,4.875184,0.000000,1.80000,3.5000,6.30000,91.295
Depth Seismic Stations,7097.0,275.364098,162.141631,0.000000,146.00000,255.0000,384.00000,934.000
Magnitude,23412.0,5.882531,0.423066,5.500000,5.60000,5.7000,6.00000,9.100
Magnitude Error,327.0,0.071820,0.051466,0.000000,0.04600,0.0590,0.07550,0.410
Magnitude Seismic Stations,2564.0,48.944618,62.943106,0.000000,10.00000,28.0000,66.00000,821.000
Azimuthal Gap,7299.0,44.163532,32.141486,0.000000,24.10000,36.0000,54.00000,360.000
Horizontal Distance,1604.0,3.992660,5.377262,0.004505,0.96875,2.3195,4.72450,37.874


In [ ]:
data.drop(['Date', 'Time', 'Depth Error', 'Depth Seismic Stations', 'Magnitude Error', 'Magnitude Seismic Stations',
           'Azimuthal Gap', 'Horizontal Distance', 'Horizontal Error', 'ID'], axis=1, inplace=True)

In [ ]:
set(data.columns) - set(['Date', 'Time', 'Depth Error', 'Depth Seismic Stations', 'Magnitude Error', 'Magnitude Seismic Stations',
           'Azimuthal Gap', 'Horizontal Distance', 'Horizontal Error', 'ID'])

{'Depth',
 'Latitude',
 'Location Source',
 'Longitude',
 'Magnitude',
 'Magnitude Source',
 'Magnitude Type',
 'Root Mean Square',
 'Source',
 'Status',
 'Type'}

In [ ]:
data_no_na = fillNA(data)

100%|██████████| 5/5 [00:00<00:00, 1272.70it/s]


In [ ]:
data_no_na

,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,Root Mean Square,Source,Location Source,Magnitude Source,Status
0,19.2460,145.6160,Earthquake,131.60,6.0,MW,1.0000,ISCGEM,ISCGEM,ISCGEM,Automatic
1,1.8630,127.3520,Earthquake,80.00,5.8,MW,1.0000,ISCGEM,ISCGEM,ISCGEM,Automatic
2,-20.5790,-173.9720,Earthquake,20.00,6.2,MW,1.0000,ISCGEM,ISCGEM,ISCGEM,Automatic
3,-59.0760,-23.5570,Earthquake,15.00,5.8,MW,1.0000,ISCGEM,ISCGEM,ISCGEM,Automatic
4,11.9380,126.4270,Earthquake,15.00,5.8,MW,1.0000,ISCGEM,ISCGEM,ISCGEM,Automatic
...,...,...,...,...,...,...,...,...,...,...,...
23407,38.3917,-118.8941,Earthquake,12.30,5.6,ML,0.1898,NN,NN,NN,Reviewed
23408,38.3777,-118.8957,Earthquake,8.80,5.5,ML,0.2187,NN,NN,NN,Reviewed
23409,36.9179,140.4262,Earthquake,10.00,5.9,MWW,1.5200,US,US,US,Reviewed
23410,-9.0283,118.6639,Earthquake,79.00,6.3,MWW,1.4300,US,US,US,Reviewed


In [ ]:
cols = data_no_na.columns
num_cols = data_no_na._get_numeric_data().columns
categorical_ftrs = list(set(cols) - set(num_cols))

data_encoded_ftrs = encode_ftrs(categorical_ftrs, data_no_na)

1 columns were label encoded.
5 columns were one-hot encoded.


In [ ]:
data_encoded_ftrs

,Latitude,Longitude,Depth,Magnitude,Root Mean Square,Status,Type_Earthquake,Type_Explosion,Type_Nuclear Explosion,Type_Rock Burst,...,Magnitude Source_NN,Magnitude Source_OFFICIAL,Magnitude Source_PAR,Magnitude Source_PGC,Magnitude Source_PR,Magnitude Source_SE,Magnitude Source_US,Magnitude Source_US_GCMT,Magnitude Source_US_PGC,Magnitude Source_UW
0,19.2460,145.6160,131.60,6.0,1.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.8630,127.3520,80.00,5.8,1.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-20.5790,-173.9720,20.00,6.2,1.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-59.0760,-23.5570,15.00,5.8,1.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11.9380,126.4270,15.00,5.8,1.0000,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,38.3917,-118.8941,12.30,5.6,0.1898,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23408,38.3777,-118.8957,8.80,5.5,0.2187,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23409,36.9179,140.4262,10.00,5.9,1.5200,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
23410,-9.0283,118.6639,79.00,6.3,1.4300,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
test = data_encoded_ftrs[::100]

In [ ]:
test.to_csv("test_nn.csv", index=False)

In [ ]:
#del data
#del data_no_na

X = data_encoded_ftrs.drop(["Magnitude", "Depth"], axis=1)
y = data_encoded_ftrs["Magnitude"]

#del data_encoded_ftrs

In [ ]:
X["Date"] = data["Date"]
X["Time"] = data["Time"]

,Latitude,Longitude,Root Mean Square,Status,Type_Earthquake,Type_Explosion,Type_Nuclear Explosion,Type_Rock Burst,Magnitude Type_MB,Magnitude Type_MD,...,Magnitude Source_NN,Magnitude Source_OFFICIAL,Magnitude Source_PAR,Magnitude Source_PGC,Magnitude Source_PR,Magnitude Source_SE,Magnitude Source_US,Magnitude Source_US_GCMT,Magnitude Source_US_PGC,Magnitude Source_UW
0,19.2460,145.6160,1.0000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1.8630,127.3520,1.0000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-20.5790,-173.9720,1.0000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-59.0760,-23.5570,1.0000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11.9380,126.4270,1.0000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23407,38.3917,-118.8941,0.1898,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23408,38.3777,-118.8957,0.2187,1,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
23409,36.9179,140.4262,1.5200,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
23410,-9.0283,118.6639,1.4300,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
y

0        6.0
1        5.8
2        6.2
3        5.8
4        5.8
        ... 
23407    5.6
23408    5.5
23409    5.9
23410    6.3
23411    5.5
Name: Magnitude, Length: 23412, dtype: float64

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X

array([[ 0.58337712,  0.84436817, -0.10383949, ..., -0.01132059,
        -0.00653567, -0.01601076],
       [ 0.00610931,  0.69884905, -0.10383949, ..., -0.01132059,
        -0.00653567, -0.01601076],
       [-0.7391616 , -1.70196151, -0.10383949, ..., -0.01132059,
        -0.00653567, -0.01601076],
       ...,
       [ 1.17023895,  0.80301824,  3.09376734, ..., -0.01132059,
        -0.00653567, -0.01601076],
       [-0.35557722,  0.62962628,  2.54033539, ..., -0.01132059,
        -0.00653567, -0.01601076],
       [ 1.18615923,  0.81085909, -0.65727144, ..., -0.01132059,
        -0.00653567, -0.01601076]])

In [ ]:
mask = np.isnan(X)
idx = np.where(~mask,np.arange(mask.shape[1]),0)
np.maximum.accumulate(idx,axis=1, out=idx)
X = X[np.arange(idx.shape[0])[:,None], idx]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_ds, test_ds = create_tensor_ds(pd.DataFrame(X_train), y_train, pd.DataFrame(X_test), y_test)

In [ ]:
train_loader_m = DataLoader(dataset = train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader_m  = DataLoader(dataset = test_ds,  batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_m = train_model_m([500], NN, device)

MSE: 34.83252967864163
MSE: 17.952889967596683
MSE: 5.750069746303244
MSE: 1.1633744791358687
MSE: 1.0438470271319886
MSE: 1.2309964075554107
MSE: 1.3433117332246858
MSE: 1.4294670655958597
MSE: 1.4754594460609711
MSE: 1.5037092595973365
MSE: 1.5065447249459105
MSE: 1.498931280715406
MSE: 1.4900086362870775
MSE: 1.4634389754466048
MSE: 1.4402571719566737
MSE: 1.4085434716709602
MSE: 1.383018426486781
MSE: 1.352932254202775
MSE: 1.3133190571911149
MSE: 1.2783947038344423
MSE: 1.2460269579099366
MSE: 1.202838208477095
MSE: 1.1824463546065471
MSE: 1.1422759651425731
MSE: 1.1092940476603022
MSE: 1.075707236250951
MSE: 1.0547724522623834
MSE: 1.001958226685323
MSE: 0.9800037226153311
MSE: 0.9277595568295426
MSE: 0.9299498751712211
MSE: 0.860928813303137
MSE: 0.8757507107666295
MSE: 0.7848577954663983
MSE: 0.8243787359379252
MSE: 0.7741312651844711
MSE: 0.7558746480654164
MSE: 0.7072991327978346
MSE: 0.6901707180511143
MSE: 0.6577749411122341
MSE: 0.6576623406302732
MSE: 0.6093460969725151
M

In [ ]:
proba, true = get_predictions(test_loader_m, model_m, device=device)
mse = metrics.mean_squared_error(y_test, true)
mae = metrics.mean_absolute_error(y_test, true)

In [ ]:
print(mse, mae)

0.3546778752288225 0.4236387154650235


In [ ]:
torch.save(model_m, "NNMagnitude.pth")

In [ ]:
X = data_encoded_ftrs.drop(["Magnitude", "Depth"], axis=1)
y = data_encoded_ftrs["Depth"]

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
mask = np.isnan(X)
idx = np.where(~mask,np.arange(mask.shape[1]),0)
np.maximum.accumulate(idx,axis=1, out=idx)
X = X[np.arange(idx.shape[0])[:,None], idx]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_ds, test_ds = create_tensor_ds(pd.DataFrame(X_train), y_train, pd.DataFrame(X_test), y_test)

In [ ]:
train_loader_d = DataLoader(dataset = train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader_d  = DataLoader(dataset = test_ds,  batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
N_EPOCHS=100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_depth = train_model_d([500], NN, device)

MSE: 19927.53022332271
MSE: 19666.837070916663
MSE: 19208.396123381935
MSE: 18467.573851858655
MSE: 17498.105916565608
MSE: 16441.9173461103
MSE: 15492.844173721474
MSE: 14774.198100886035
MSE: 14331.521304417902
MSE: 14115.65802408084
MSE: 14008.71911789116
MSE: 13953.992735369322
MSE: 13922.887723626885
MSE: 13894.643373210953
MSE: 13873.200063914852
MSE: 13852.895172583905
MSE: 13836.158574221769
MSE: 13819.229716877267
MSE: 13806.451008491456
MSE: 13791.284316875506
MSE: 13778.900034164459
MSE: 13766.195007889686
MSE: 13754.813841276735
MSE: 13742.384064252925
MSE: 13733.23844138053
MSE: 13721.899058625911
MSE: 13710.185432716813
MSE: 13701.634633284824
MSE: 13690.81341511687
MSE: 13679.924123347524
MSE: 13670.702269455718
MSE: 13660.436089224673
MSE: 13652.054529219802
MSE: 13643.636377385334
MSE: 13634.906172913685
MSE: 13630.464905172676
MSE: 13623.174409061225
MSE: 13615.21675419272
MSE: 13613.408302404961
MSE: 13610.782355924488
MSE: 13608.427486493352
MSE: 13605.277025738158


In [ ]:
proba, true = get_predictions(test_loader_d, model_depth, device=device)
mse = metrics.mean_squared_error(y_test, true)
mae = metrics.mean_absolute_error(y_test, true)

In [ ]:
print(mse, mae)

29933.760590350263 88.7195024817527


In [ ]:
torch.save(model_depth, "NNDepth.pth")

In [ ]:
def predict(_models, _loaders):
        predictions = []
        for model, loader in zip(_models, _loaders):
            predict, true  = get_predictions(loader, model, device)
            predictions.append(predict)
        print(predictions)

        return np.reshape(np.array(predictions), (-1, 4683))

In [ ]:
mdl_d = torch.load('NNDepth.pth')
mdl_m = torch.load('NNMagnitude.pth')
_models = [mdl_d, mdl_m]
_loaders = [test_loader_d, test_loader_m]

In [ ]:
def get_predictions(loader, model, device):
  model.eval()
  saved_preds = []
  true_labels = []
  
  with torch.no_grad():
      for x,y in loader:
          x = x.to(device)
          y = y.to(device)
          scores = model(x)
          saved_preds += scores.tolist()
          true_labels += y.tolist()
  return saved_preds, true_labels

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
d_m = predict(_models, _loaders)

[[83.36018371582031, 91.82798767089844, 24.07297706604004, 17.534637451171875, 62.48393630981445, 66.79157257080078, 48.14171600341797, 79.74275970458984, 83.52371215820312, 39.631717681884766, 80.8455581665039, 32.093074798583984, 21.55324935913086, 59.05820083618164, 71.17707824707031, 82.20649719238281, 101.42668151855469, 50.777103424072266, 77.50538635253906, 95.76651763916016, 38.25465774536133, -5.561857223510742, 115.68340301513672, 102.1212158203125, 55.45321273803711, 69.8861083984375, 24.493427276611328, 19.506635665893555, 70.49624633789062, 47.37234878540039, 61.6065788269043, -12.03274917602539, 86.53068542480469, 88.55448150634766, 136.55010986328125, 64.39604949951172, 113.57042694091797, 57.33430862426758, 43.65140914916992, 54.727657318115234, 83.52255249023438, 77.79277038574219, 69.54219818115234, 48.33556365966797, 79.74644470214844, 79.80816650390625, 47.75090408325195, 102.2213134765625, 105.3796615600586, 51.21742630004883, 106.97390747070312, 47.35023498535156,

In [ ]:
d_m.shape

(2, 4683)

In [ ]:
X_test.shape

(4683, 103)

In [ ]:
d_m

array([[83.36018372, 91.82798767, 24.07297707, ..., 27.09877205,
        64.19989014, 73.22068787],
       [ 6.04629993,  5.87114716,  5.65208435, ...,  5.79710007,
         5.91515636,  6.11124992]])

In [ ]:
print(len(train_loader_m.dataset))

18729
